In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

## uploading daily temperatures 

In [3]:
years = ['2000', '2001', '2002', '2003', '2004','2005', '2006', '2007', '2008']

In [4]:
frames = []
for year in years:
    ur_files = ddf.read_csv('DailyTemp/DailyTemp_' + year + '.csv', dtype = {'fips': str, 'STATION': 'object'})
    temp = ur_files.compute()
    temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
    temp['year'] = year
    frames.append(temp)

/global/homes/w/wenhuan/.conda/envs/myenv/lib/python3.7/site-packages/dask/dataframe/io/csv.py:134: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  path_info,


In [5]:
temp = pd.concat(frames)

In [6]:
temp.head()

,STATION,LATITUDE,LONGITUDE,NAME,YEAR,DATE,TEMP,MIN,MAX,year
0,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-01,31.4,27.3,34.2,2000
1,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-02,29.0,26.8,30.4,2000
2,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-03,28.1,26.8,30.2,2000
3,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-04,27.7,26.6,29.5,2000
4,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-05,28.2,23.4,30.4,2000


In [7]:
temp.shape

(26615983, 10)

### uploading temp_station data

In [8]:
frames = []
for year in years:
    ur_files = ddf.read_csv('temp_station/temp_station_' + year + '.csv', dtype = {'STATEFP': str, 'COUNTYFP': str, 'TRACTCE': 'object', 'station': str})
    temp_station = ur_files.compute()
    temp_station = temp_station.loc[:, ~temp_station.columns.str.contains('^Unnamed')]
    frames.append(temp_station)

In [9]:
temp_station = pd.concat(frames)
temp_station.head()

,STATEFP,COUNTYFP,TRACTCE,station,year
0,13,163,960300,72218003820,2000.0
1,37,109,071201,72314013881,2000.0
2,25,025,050200,72509014739,2000.0
3,17,031,230200,72534699999,2000.0
4,41,047,001607,72694024232,2000.0


In [10]:
temp_station.shape

(767070, 5)

In [11]:
temp_station = temp_station.dropna()
temp_station.shape

(749209, 5)

In [12]:
temp_station['year'] = temp_station['year'].astype(int)
temp_station['year'].dtype

dtype('int64')

In [13]:
temp_station.shape

(749209, 5)

### Merge temp_station and temperature data

In [14]:
temp = temp.rename(columns = {'STATION': 'station'})

In [15]:
temp['year'] = temp['year'].astype(int)

In [16]:
temp.head()

,station,LATITUDE,LONGITUDE,NAME,YEAR,DATE,TEMP,MIN,MAX,year
0,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-01,31.4,27.3,34.2,2000
1,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-02,29.0,26.8,30.4,2000
2,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-03,28.1,26.8,30.2,2000
3,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-04,27.7,26.6,29.5,2000
4,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-05,28.2,23.4,30.4,2000


In [17]:
temp_station.head()

,STATEFP,COUNTYFP,TRACTCE,station,year
0,13,163,960300,72218003820,2000
1,37,109,071201,72314013881,2000
2,25,025,050200,72509014739,2000
3,17,031,230200,72534699999,2000
4,41,047,001607,72694024232,2000


In [18]:
temp['station'].dtype

dtype('O')

In [19]:
temp_station['station'].dtype

dtype('O')

In [20]:
temp['year'].dtype

dtype('int64')

In [21]:
temp_station['year'].dtype

dtype('int64')

## 2000

In [22]:
temp_2000 = temp[temp['year'] == 2000]
temp_station_2000 = temp_station[temp_station['year'] == 2000]

In [23]:
temp_2000.shape

(2546974, 10)

In [24]:
temp_station_2000.shape

(82143, 5)

In [25]:
temp_station_2000 = temp_2000.merge(temp_station_2000, on = ['station', 'year'], how = 'left')
temp_station_2000.head()

,station,LATITUDE,LONGITUDE,NAME,YEAR,DATE,TEMP,MIN,MAX,year,STATEFP,COUNTYFP,TRACTCE
0,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-01,31.4,27.3,34.2,2000,NaN,NaN,NaN
1,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-02,29.0,26.8,30.4,2000,NaN,NaN,NaN
2,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-03,28.1,26.8,30.2,2000,NaN,NaN,NaN
3,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-04,27.7,26.6,29.5,2000,NaN,NaN,NaN
4,1001099999,70.933333,-8.666667,"JAN MAYEN NOR NAVY, NO",2000,2000-01-05,28.2,23.4,30.4,2000,NaN,NaN,NaN


In [26]:
temp_station_2000 = temp_station_2000.dropna()
temp_station_2000.head()

,station,LATITUDE,LONGITUDE,NAME,YEAR,DATE,TEMP,MIN,MAX,year,STATEFP,COUNTYFP,TRACTCE
1397981,69015093121,34.3,-116.16667,"TWENTYNINE PALMS MC, CA US",2000,2000-01-03,48.2,35.6,55.4,2000,06,071,010434
1397982,69015093121,34.3,-116.16667,"TWENTYNINE PALMS MC, CA US",2000,2000-01-03,48.2,35.6,55.4,2000,06,071,010433
1397983,69015093121,34.3,-116.16667,"TWENTYNINE PALMS MC, CA US",2000,2000-01-03,48.2,35.6,55.4,2000,06,071,010422
1397984,69015093121,34.3,-116.16667,"TWENTYNINE PALMS MC, CA US",2000,2000-01-03,48.2,35.6,55.4,2000,06,071,010432
1397985,69015093121,34.3,-116.16667,"TWENTYNINE PALMS MC, CA US",2000,2000-01-03,48.2,35.6,55.4,2000,06,071,010410


In [27]:
temp_station_2000.shape

(29300336, 13)

In [28]:
temp_station_final = temp_station_2000.groupby(['STATEFP', 'COUNTYFP', 'DATE']).mean()
temp_station_final = temp_station_final.reset_index()
temp_station_final.head()

,STATEFP,COUNTYFP,DATE,LATITUDE,LONGITUDE,YEAR,TEMP,MIN,MAX,year
0,01,001,2000-01-01,32.2997,-86.407500,2000.0,61.000000,50.000000,75.900000,2000.0
1,01,001,2000-01-02,32.3781,-86.369382,2000.0,69.500000,61.923529,75.241176,2000.0
2,01,001,2000-01-03,32.3781,-86.369382,2000.0,72.429412,64.317647,80.388235,2000.0
3,01,001,2000-01-04,32.3781,-86.369382,2000.0,63.876471,51.576471,73.611765,2000.0
4,01,001,2000-01-05,32.3781,-86.369382,2000.0,43.729412,32.000000,51.758824,2000.0


In [29]:
temp_station_final['fips'] = temp_station_final['STATEFP'] + temp_station_final['COUNTYFP'] 
temp_station_final = temp_station_final.rename(columns = {'DATE': 'date', 'TEMP': 'temp', 'MIN': 'min_temp', 'MAX': 'max_temp'})
temp_station_final = temp_station_final[['fips', 'date', 'temp', 'min_temp', 'max_temp', 'year']]
temp_station_final.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2000-01-01,61.000000,50.000000,75.900000,2000.0
1,01001,2000-01-02,69.500000,61.923529,75.241176,2000.0
2,01001,2000-01-03,72.429412,64.317647,80.388235,2000.0
3,01001,2000-01-04,63.876471,51.576471,73.611765,2000.0
4,01001,2000-01-05,43.729412,32.000000,51.758824,2000.0


In [30]:
temp_station_final.shape

(1057314, 6)

In [31]:
len(temp_station_final['fips'].unique())

2925

In [32]:
temp_station_final.to_csv('daily_temp_county_level_2000.csv')

### 2001

In [33]:
temp_2001 = temp[temp['year'] == 2001]
temp_station_2001 = temp_station[temp_station['year'] == 2001]
temp_station_2001 = temp_2001.merge(temp_station_2001, on = ['station', 'year'], how = 'left')
temp_station_2001 = temp_station_2001.dropna()
temp_station_final = temp_station_2001.groupby(['STATEFP', 'COUNTYFP', 'DATE']).mean()
temp_station_final = temp_station_final.reset_index()
temp_station_final['fips'] = temp_station_final['STATEFP'] + temp_station_final['COUNTYFP'] 
temp_station_final = temp_station_final.rename(columns = {'DATE': 'date', 'TEMP': 'temp', 'MIN': 'min_temp', 'MAX': 'max_temp'})
temp_station_final = temp_station_final[['fips', 'date', 'temp', 'min_temp', 'max_temp', 'year']]
temp_station_final.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2001-01-01,32.100000,17.100000,41.000000,2001.0
1,01001,2001-01-02,30.970588,19.200000,42.694118,2001.0
2,01001,2001-01-03,30.488235,20.841176,42.758824,2001.0
3,01001,2001-01-04,36.876471,24.347059,49.882353,2001.0
4,01001,2001-01-05,42.917647,24.629412,60.758824,2001.0


In [34]:
temp_station_final.to_csv('daily_temp_county_level_2001.csv')

### 2012

In [35]:
temp_2002 = temp[temp['year'] == 2002]
temp_station_2002 = temp_station[temp_station['year'] == 2002]
temp_station_2002 = temp_2002.merge(temp_station_2002, on = ['station', 'year'], how = 'left')
temp_station_2002 = temp_station_2002.dropna()
temp_station_final = temp_station_2002.groupby(['STATEFP', 'COUNTYFP', 'DATE']).mean()
temp_station_final = temp_station_final.reset_index()
temp_station_final['fips'] = temp_station_final['STATEFP'] + temp_station_final['COUNTYFP'] 
temp_station_final = temp_station_final.rename(columns = {'DATE': 'date', 'TEMP': 'temp', 'MIN': 'min_temp', 'MAX': 'max_temp'})
temp_station_final = temp_station_final[['fips', 'date', 'temp', 'min_temp', 'max_temp', 'year']]
temp_station_final.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2002-01-01,32.600000,21.900000,42.100000,2002.0
1,01001,2002-01-02,35.852941,33.111765,37.676471,2002.0
2,01001,2002-01-03,35.100000,33.694118,37.317647,2002.0
3,01001,2002-01-04,33.864706,21.070588,48.011765,2002.0
4,01001,2002-01-05,41.276471,22.705882,58.935294,2002.0


In [36]:
temp_station_final.to_csv('daily_temp_county_level_2002.csv')

### 2013

In [37]:
temp_2003 = temp[temp['year'] == 2003]
temp_station_2003 = temp_station[temp_station['year'] == 2003]
temp_station_2003 = temp_2003.merge(temp_station_2003, on = ['station', 'year'], how = 'left')
temp_station_2003 = temp_station_2003.dropna()
temp_station_final = temp_station_2003.groupby(['STATEFP', 'COUNTYFP', 'DATE']).mean()
temp_station_final = temp_station_final.reset_index()
temp_station_final['fips'] = temp_station_final['STATEFP'] + temp_station_final['COUNTYFP'] 
temp_station_final = temp_station_final.rename(columns = {'DATE': 'date', 'TEMP': 'temp', 'MIN': 'min_temp', 'MAX': 'max_temp'})
temp_station_final = temp_station_final[['fips', 'date', 'temp', 'min_temp', 'max_temp', 'year']]
temp_station_final.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2003-01-01,53.900000,50.000000,61.000000,2003.0
1,01001,2003-01-02,54.100000,48.094118,66.070588,2003.0
2,01001,2003-01-03,41.129412,37.376471,48.188235,2003.0
3,01001,2003-01-04,40.300000,28.264706,51.758824,2003.0
4,01001,2003-01-05,50.888235,35.047059,64.376471,2003.0


In [38]:
temp_station_final.to_csv('daily_temp_county_level_2003.csv')

### 2004

In [39]:
temp_2004 = temp[temp['year'] == 2004]
temp_station_2004 = temp_station[temp_station['year'] == 2004]
temp_station_2004 = temp_2004.merge(temp_station_2004, on = ['station', 'year'], how = 'left')
temp_station_2004 = temp_station_2004.dropna()
temp_station_final = temp_station_2004.groupby(['STATEFP', 'COUNTYFP', 'DATE']).mean()
temp_station_final = temp_station_final.reset_index()
temp_station_final['fips'] = temp_station_final['STATEFP'] + temp_station_final['COUNTYFP'] 
temp_station_final = temp_station_final.rename(columns = {'DATE': 'date', 'TEMP': 'temp', 'MIN': 'min_temp', 'MAX': 'max_temp'})
temp_station_final = temp_station_final[['fips', 'date', 'temp', 'min_temp', 'max_temp', 'year']]
temp_station_final.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2004-01-01,43.676471,38.670588,51.123529,2004.0
1,01001,2004-01-02,54.100000,32.000000,69.100000,2004.0
2,01001,2004-01-03,71.394118,61.447059,77.000000,2004.0
3,01001,2004-01-04,71.076471,62.388235,78.694118,2004.0
4,01001,2004-01-05,61.311765,49.788235,69.876471,2004.0


In [40]:
temp_station_final.to_csv('daily_temp_county_level_2004.csv')

### 2005

In [41]:
temp_2005 = temp[temp['year'] == 2005]
temp_station_2005 = temp_station[temp_station['year'] == 2005]
temp_station_2005 = temp_2005.merge(temp_station_2005, on = ['station', 'year'], how = 'left')
temp_station_2005 = temp_station_2005.dropna()
temp_station_final = temp_station_2005.groupby(['STATEFP', 'COUNTYFP', 'DATE']).mean()
temp_station_final = temp_station_final.reset_index()
temp_station_final['fips'] = temp_station_final['STATEFP'] + temp_station_final['COUNTYFP'] 
temp_station_final = temp_station_final.rename(columns = {'DATE': 'date', 'TEMP': 'temp', 'MIN': 'min_temp', 'MAX': 'max_temp'})
temp_station_final = temp_station_final[['fips', 'date', 'temp', 'min_temp', 'max_temp', 'year']]
temp_station_final.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2005-01-01,60.100000,48.900000,75.000000,2005.0
1,01001,2005-01-02,67.129412,51.717647,75.541176,2005.0
2,01001,2005-01-03,63.811765,50.000000,75.541176,2005.0
3,01001,2005-01-04,59.841176,48.176471,70.776471,2005.0
4,01001,2005-01-05,60.564706,48.282353,73.694118,2005.0


In [42]:
temp_station_final.to_csv('daily_temp_county_level_2005.csv')

### 2006

In [22]:
temp_2006 = temp[temp['year'] == 2006]
temp_station_2006 = temp_station[temp_station['year'] == 2006]
temp_station_2006 = temp_2006.merge(temp_station_2006, on = ['station', 'year'], how = 'left')
temp_station_2006 = temp_station_2006.dropna()
temp_station_final = temp_station_2006.groupby(['STATEFP', 'COUNTYFP', 'DATE']).mean()
temp_station_final = temp_station_final.reset_index()
temp_station_final['fips'] = temp_station_final['STATEFP'] + temp_station_final['COUNTYFP'] 
temp_station_final = temp_station_final.rename(columns = {'DATE': 'date', 'TEMP': 'temp', 'MIN': 'min_temp', 'MAX': 'max_temp'})
temp_station_final = temp_station_final[['fips', 'date', 'temp', 'min_temp', 'max_temp', 'year']]
temp_station_final.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2006-01-01,56.600000,48.900000,69.100000,2006.0
1,01001,2006-01-02,68.900000,48.900000,81.000000,2006.0
2,01001,2006-01-03,57.723529,49.764706,66.352941,2006.0
3,01001,2006-01-04,57.147059,40.176471,73.235294,2006.0
4,01001,2006-01-05,55.541176,48.117647,60.529412,2006.0


In [23]:
temp_station_final.to_csv('daily_temp_county_level_2006.csv')

### 2007

In [24]:
temp_2007 = temp[temp['year'] == 2007]
temp_station_2007 = temp_station[temp_station['year'] == 2007]
temp_station_2007 = temp_2007.merge(temp_station_2007, on = ['station', 'year'], how = 'left')
temp_station_2007 = temp_station_2007.dropna()
temp_station_final = temp_station_2007.groupby(['STATEFP', 'COUNTYFP', 'DATE']).mean()
temp_station_final = temp_station_final.reset_index()
temp_station_final['fips'] = temp_station_final['STATEFP'] + temp_station_final['COUNTYFP'] 
temp_station_final = temp_station_final.rename(columns = {'DATE': 'date', 'TEMP': 'temp', 'MIN': 'min_temp', 'MAX': 'max_temp'})
temp_station_final = temp_station_final[['fips', 'date', 'temp', 'min_temp', 'max_temp', 'year']]
temp_station_final.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2007-01-01,53.600000,46.900000,72.000000,2007.0
1,01001,2007-01-02,43.700000,30.000000,57.900000,2007.0
2,01001,2007-01-03,49.984615,36.846154,57.253846,2007.0
3,01001,2007-01-04,59.923077,51.746154,66.253846,2007.0
4,01001,2007-01-05,64.684615,60.746154,68.138462,2007.0


In [25]:
temp_station_final.to_csv('daily_temp_county_level_2007.csv')

### 2008

In [26]:
temp_2008 = temp[temp['year'] == 2008]
temp_station_2008 = temp_station[temp_station['year'] == 2008]
temp_station_2008 = temp_2008.merge(temp_station_2008, on = ['station', 'year'], how = 'left')
temp_station_2008 = temp_station_2008.dropna()
temp_station_final = temp_station_2008.groupby(['STATEFP', 'COUNTYFP', 'DATE']).mean()
temp_station_final = temp_station_final.reset_index()
temp_station_final['fips'] = temp_station_final['STATEFP'] + temp_station_final['COUNTYFP'] 
temp_station_final = temp_station_final.rename(columns = {'DATE': 'date', 'TEMP': 'temp', 'MIN': 'min_temp', 'MAX': 'max_temp'})
temp_station_final = temp_station_final[['fips', 'date', 'temp', 'min_temp', 'max_temp', 'year']]
temp_station_final.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2008-01-01,43.917647,37.900000,60.217647,2008.0
1,01001,2008-01-02,32.552941,26.152941,44.758824,2008.0
2,01001,2008-01-03,27.805882,17.541176,41.276471,2008.0
3,01001,2008-01-04,32.411765,19.823529,50.782353,2008.0
4,01001,2008-01-05,44.058824,30.847059,63.594118,2008.0


In [27]:
temp_station_final.to_csv('daily_temp_county_level_2008.csv')

### 2019

In [33]:
temp_2019 = temp[temp['year'] == 2019]
temp_station_2019 = temp_station[temp_station['year'] == 2019]
temp_station_2019 = temp_2019.merge(temp_station_2019, on = ['station', 'year'], how = 'left')
temp_station_2019 = temp_station_2019.dropna()
temp_station_final = temp_station_2019.groupby(['STATEFP', 'COUNTYFP', 'DATE']).mean()
temp_station_final = temp_station_final.reset_index()
temp_station_final['fips'] = temp_station_final['STATEFP'] + temp_station_final['COUNTYFP'] 
temp_station_final = temp_station_final.rename(columns = {'DATE': 'date', 'TEMP': 'temp', 'MIN': 'min_temp', 'MAX': 'max_temp'})
temp_station_final = temp_station_final[['fips', 'date', 'temp', 'min_temp', 'max_temp', 'year']]
temp_station_final.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2019-01-01,65.905882,61.447059,71.770588,2019.0
1,01001,2019-01-02,58.700000,55.941176,62.652941,2019.0
2,01001,2019-01-03,56.229412,53.964706,59.347059,2019.0
3,01001,2019-01-04,57.547059,48.635294,63.958824,2019.0
4,01001,2019-01-05,49.741176,42.541176,62.058824,2019.0


In [34]:
temp_station_final.to_csv('daily_temp_county_level_2019.csv')

### 2020

In [22]:
temp_2020 = temp[temp['year'] == 2020]
temp_station_2020 = temp_station[temp_station['year'] == 2020]
temp_station_2020 = temp_2020.merge(temp_station_2020, on = ['station', 'year'], how = 'left')
temp_station_2020 = temp_station_2020.dropna()
temp_station_final = temp_station_2020.groupby(['STATEFP', 'COUNTYFP', 'DATE']).mean()
temp_station_final = temp_station_final.reset_index()
temp_station_final['fips'] = temp_station_final['STATEFP'] + temp_station_final['COUNTYFP'] 
temp_station_final = temp_station_final.rename(columns = {'DATE': 'date', 'TEMP': 'temp', 'MIN': 'min_temp', 'MAX': 'max_temp'})
temp_station_final = temp_station_final[['fips', 'date', 'temp', 'min_temp', 'max_temp', 'year']]
temp_station_final.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2020-01-01,42.800000,30.729412,57.023529,2020.0
1,01001,2020-01-02,51.723529,43.970588,64.241176,2020.0
2,01001,2020-01-03,65.823529,59.435294,71.370588,2020.0
3,01001,2020-01-04,57.464706,49.864706,61.829412,2020.0
4,01001,2020-01-05,43.417647,30.694118,57.535294,2020.0


In [23]:
temp_station_final.to_csv('daily_temp_county_level_2020.csv')

### 2009

In [45]:
temp_2009 = temp[temp['year'] == 2009]
temp_station_2009 = temp_station[temp_station['year'] == 2009]
temp_station_2009 = temp_2009.merge(temp_station_2009, on = ['station', 'year'], how = 'left')
temp_station_2009 = temp_station_2009.dropna()
temp_station_final = temp_station_2009.groupby(['STATEFP', 'COUNTYFP', 'DATE']).mean()
temp_station_final = temp_station_final.reset_index()
temp_station_final['fips'] = temp_station_final['STATEFP'] + temp_station_final['COUNTYFP'] 
temp_station_final = temp_station_final.rename(columns = {'DATE': 'date', 'TEMP': 'temp', 'MIN': 'min_temp', 'MAX': 'max_temp'})
temp_station_final = temp_station_final[['fips', 'date', 'temp', 'min_temp', 'max_temp', 'year']]
temp_station_final.head()

,fips,date,temp,min_temp,max_temp,year
0,01001,2009-01-01,41.323529,34.311765,49.282353,2009.0
1,01001,2009-01-02,51.400000,44.520000,61.340000,2009.0
2,01001,2009-01-03,65.670588,58.135294,72.923529,2009.0
3,01001,2009-01-04,64.529412,59.023529,69.358824,2009.0
4,01001,2009-01-05,63.382353,60.852941,66.270588,2009.0


In [46]:
temp_station_final.to_csv('daily_temp_county_level_2009.csv')